In [ ]:
!unzip /content/drive/MyDrive/train.zip -d /content/train

In [2]:
%%capture
!git clone https://github.com/huggingface/diffusers
%cd diffusers
!pip install .

In [3]:
%%capture
%cd /content/diffusers/examples/unconditional_image_generation
!pip install -r requirements.txt

In [ ]:
!pip install accelerate

In [ ]:
!accelerate config default
!mv /content/default_config.yaml /root/.cache/huggingface/accelerate/
!accelerate config update

In [6]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid.
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [7]:
import os
import glob
import shutil
import pandas as pd

In [8]:
files = glob.glob("/content/train/*/*/*.webp")
os.makedirs("/content/train_data", exist_ok=True)

for f in files:
    shutil.copy2(f, f"/content/train_data/{f.split('/')[-1]}")

In [ ]:
!accelerate launch --mixed_precision="fp16" train_unconditional.py \
  --train_data_dir="/content/train_data" \
  --resolution=64 --random_flip --center_crop \
  --train_batch_size=16 \
  --num_epochs=100 --checkpointing_steps=5000 \
  --learning_rate=1e-04 --lr_scheduler="constant" --lr_warmup_steps=500 \
  --output_dir="/content/output" 

2023-05-02 08:28:27.806651: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-02 08:28:28.691764: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-05-02 08:28:33.721586: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:249: FutureWarning: `logging_dir` is deprecated and will be removed in version 0.18.0 of 🤗 Accelerate. Use `project_dir` instead.
  warnings.warn(
05/02/2023 08:28:37 - INFO - __main__ - Distributed environment: NO
Num processes: 1
Process index: 0
Local process index: 0
Device: cuda

Mixed precision type: no

Resolving data files: 100% 120/120 [00:00<00:00, 18808

In [15]:
from diffusers import DDIMPipeline, DDIMScheduler

device = "cuda"

# load model and scheduler
pipeline = DDIMPipeline.from_pretrained('./output-256/epoch-10', local_files_only=True)
pipeline.scheduler = DDIMScheduler.from_config(pipeline.scheduler.config)
pipeline.to(device)

# generate image
image = pipeline(num_inference_steps=50).images[0]
# save image
image.save("image.png")

The config attributes {'feature_extractor': ['transformers', 'CLIPFeatureExtractor'], 'safety_checker': ['stable_diffusion', 'StableDiffusionSafetyChecker'], 'text_encoder': ['transformers', 'CLIPTextModel'], 'tokenizer': ['transformers', 'CLIPTokenizer'], 'vae': ['diffusers', 'AutoencoderKL']} were passed to DDPMPipeline, but are not expected and will be ignored. Please verify your model_index.json configuration file.
Keyword arguments {'feature_extractor': ['transformers', 'CLIPFeatureExtractor'], 'safety_checker': ['stable_diffusion', 'StableDiffusionSafetyChecker'], 'text_encoder': ['transformers', 'CLIPTextModel'], 'tokenizer': ['transformers', 'CLIPTokenizer'], 'vae': ['diffusers', 'AutoencoderKL']} are not expected by DDPMPipeline and will be ignored.
Cannot initialize model with low cpu memory usage because `accelerate` was not found in the environment. Defaulting to `low_cpu_mem_usage=False`. It is strongly recommended to install `accelerate` for faster and less memory-intense

  0%|          | 0/50 [00:00<?, ?it/s]

TypeError: ignored